导入包

In [1]:
import java.io.File
import com.lignting.neural.*
import org.jetbrains.kotlinx.multik.api.mk
import org.jetbrains.kotlinx.multik.api.ndarray
import org.jetbrains.kotlinx.multik.ndarray.operations.toList
import org.jetbrains.kotlinx.multik.ndarray.data.get
import com.github.doyaaaaaken.kotlincsv.dsl.csvWriter
import com.github.doyaaaaaken.kotlincsv.dsl.csvReader

数据地址

In [2]:
val trainPath = "res/used_car_train_20200313.csv"
val testPath = "res/used_car_testA_20200313.csv"

使用kotlin-csv-jvm库导入csv数据

In [3]:
fun getDataFrame(path:String): AnyFrame {
    val dataMap = mutableMapOf<String, MutableList<Double?>>()
    csvReader {
        delimiter = ' '
    }.readAllWithHeader(File(path)).forEach {
        it.forEach { entry ->
            if (dataMap[entry.key] == null)
                dataMap[entry.key] = mutableListOf()
            dataMap[entry.key]!!.add(entry.value.toDoubleOrNull())
        }
    }

    return dataMap.toDataFrame()
}

In [4]:
var trainDf = getDataFrame(trainPath)
var targetDf = getDataFrame(testPath)

检查各行空值数量并使用-1填充空值

In [5]:
trainDf.describe().select { name and nulls }

name,nulls
SaleID,0
name,0
regDate,0
model,1
brand,0
bodyType,4506
fuelType,8680
gearbox,5981
power,0
kilometer,0


In [6]:
trainDf = trainDf.fillNulls{ model and bodyType and fuelType and gearbox and notRepairedDamage }.with { -1.0 }
targetDf = targetDf.fillNulls{ model and bodyType and fuelType and gearbox and notRepairedDamage }.with { -1.0 }

In [7]:
trainDf.describe().select { name and nulls }

name,nulls
SaleID,0
name,0
regDate,0
model,0
brand,0
bodyType,0
fuelType,0
gearbox,0
power,0
kilometer,0


分析各行情况

In [8]:
trainDf.describe()

name,type,count,unique,nulls,top,freq,mean,std,min,median,max
SaleID,Double,150000,150000,0,0.000000,1,74999.500000,43301.414527,0.000000,74999.500000,149999.000000
name,Double,150000,99662,0,708.000000,282,68349.172873,61103.875095,0.000000,51638.000000,196812.000000
regDate,Double,150000,3894,0,20000008.000000,180,20034170.512180,53649.879255,19910001.000000,20030912.000000,20151212.000000
model,Double,150000,249,0,0.000000,11762,47.128700,49.536030,-1.000000,30.000000,247.000000
brand,Double,150000,40,0,0.000000,31480,8.052733,7.864956,0.000000,6.000000,39.000000
bodyType,Double,150000,9,0,0.000000,41420,1.708487,1.798313,-1.000000,1.000000,7.000000
fuelType,Double,150000,8,0,0.000000,91656,0.296227,0.621953,-1.000000,0.000000,6.000000
gearbox,Double,150000,3,0,0.000000,111623,0.176100,0.474170,-1.000000,0.000000,1.000000
power,Double,150000,566,0,0.000000,12829,119.316547,177.168419,0.000000,110.000000,19312.000000
kilometer,Double,150000,13,0,15.000000,96877,12.597160,3.919576,0.500000,15.000000,15.000000


定义离散变量和连续变量

In [9]:
val num_cols = listOf(
    "power",
    "kilometer",
    "v_0",
    "v_1",
    "v_2",
    "v_3",
    "v_4",
    "v_5",
    "v_6",
    "v_7",
    "v_8",
    "v_9",
    "v_10",
    "v_11",
    "v_12",
    "v_13",
    "v_14"
)
val cate_cols = listOf("model", "brand", "bodyType", "fuelType")
val boolean_cols = listOf("gearbox", "seller", "notRepairedDamage", "offerType")

简易EDA

In [10]:
num_cols.map { col ->
    val data = trainDf[col].values.groupingBy { it }.eachCount()
    plot {
        line {
            x(data.keys, name = col)
            y(data.values, name = "price")
            color = Color.BLUE
        }
    }
}[0]

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="l3ZuPt"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"power",
"y":"price"
},
"stat":"identity",
"data":{
"price":[6374.0,12829.0,2746.0,1120.0,1207.0,2049.0,6495.0,5537.0,1081.0,637.0,9593.0,1597.0,160.0,484.0,425.0,1161.0,4457.0,2293.0,5177.0,2435.0,1041.0,4791.0,4890.0,495.0,281.0,2714.0,1725.0,5963.0,1316.0,291.0,2325.0,2064.0,1029.0,896.0,1660.0,322.0,386.0,33.0,2813.0,28.0,2956.0,158.0,740.0,702.0,245.0,57.0,723.0,573.0,2313.0,603.0,1485.0,380.0,115.0,657.0,90.0,959.0,1734.0,613.0,162.0,291.0,1751.0,80.0,224.0,1202.0,235.0,553.0,401.0,173.0,1249.0,1086.0,126.0,14.0,1.0,221.0,419.0,336.0,40.0,102.0,375.0,336.0,570.0,185.0,947.0,123.0,318.0,119.0,396.0,179.0,486.0,35.0,999.0,32.0,196.0,45.0,353.0,403.0,117.0,368.0,439.0,1231.0,41.0,68.0,377.0,586.0,413.0,83.0,380.0,16.0,392.0,90.0,1038.0,27.0,1.0,539.0,132.0,180.0,235.0,317.0,6.0,15.0,49.0,80.0,249.0,52.0,221.0,9.0,55.0,439.0,24.0,79.0,171.0,1.0,200.0,1.0,148.0,345.0,11.0,46.0,26.0,2.0,308.0,38.0,152.0,15.0,139.0,77.0,504.0,59.0,41.0,9.0,443.0,140.0,94.0,57.0,64.0,123.0,52.0,1.0,15.0,97.0,79.0,63.0,169.0,103.0,44.0,356.0,44.0,84.0,109.0,43.0,50.0,24.0,96.0,53.0,33.0,104.0,11.0,5.0,126.0,40.0,87.0,138.0,170.0,112.0,20.0,78.0,40.0,53.0,34.0,48.0,49.0,27.0,8.0,42.0,37.0,52.0,51.0,5.0,32.0,49.0,34.0,13.0,7.0,20.0,82.0,6.0,37.0,39.0,68.0,2.0,51.0,23.0,15.0,4.0,18.0,67.0,2.0,13.0,50.0,13.0,19.0,11.0,4.0,37.0,1.0,44.0,12.0,5.0,4.0,62.0,22.0,108.0,1.0,51.0,19.0,28.0,4.0,22.0,4.0,1.0,6.0,6.0,8.0,24.0,1.0,22.0,5.0,49.0,18.0,1.0,1.0,27.0,19.0,12.0,14.0,26.0,1.0,31.0,36.0,11.0,6.0,30.0,1.0,5.0,10.0,45.0,45.0,10.0,8.0,15.0,8.0,1.0,8.0,25.0,17.0,9.0,1.0,18.0,15.0,8.0,49.0,16.0,1.0,1.0,9.0,7.0,2.0,2.0,20.0,88.0,15.0,16.0,1.0,2.0,1.0,6.0,2.0,2.0,3.0,19.0,5.0,4.0,28.0,5.0,46.0,2.0,10.0,4.0,4.0,2.0,5.0,8.0,6.0,1.0,3.0,3.0,4.0,15.0,3.0,33.0,1.0,6.0,3.0,2.0,3.0,1.0,1.0,8.0,5.0,5.0,3.0,1.0,5.0,14.0,1.0,1.0,2.0,1.0,4.0,14.0,10.0,4.0,8.0,4.0,8.0,1.0,1.0,14.0,5.0,6.0,14.0,1.0,2.0,1.0,2.0,1.0,16.0,1.0,4.0,1.0,7.0,1.0,3.0,4.0,3.0,3.0,4.0,1.0,1.0,2.0,2.0,7.0,3.0,1.0,2.0,5.0,3.0,6.0,2.0,11.0,4.0,12.0,3.0,2.0,1.0,1.0,11.0,6.0,1.0,6.0,1.0,1.0,2.0,4.0,1.0,3.0,1.0,9.0,8.0,2.0,3.0,8.0,1.0,1.0,1.0,1.0,3.0,1.0,4.0,14.0,2.0,3.0,2.0,1.0,2.0,3.0,1.0,2.0,1.0,1.0,11.0,6.0,1.0,8.0,6.0,1.0,1.0,1.0,11.0,5.0,1.0,7.0,2.0,1.0,1.0,1.0,3.0,1.0,3.0,1.0,3.0,3.0,1.0,1.0,4.0,5.0,7.0,1.0,2.0,1.0,5.0,3.0,1.0,1.0,2.0,4.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0],
"power":[60.0,0.0,163.0,193.0,68.0,109.0,150.0,101.0,179.0,88.0,75.0,58.0,128.0,239.0,145.0,45.0,105.0,54.0,116.0,143.0,218.0,170.0,90.0,306.0,133.0,102.0,177.0,140.0,115.0,180.0,131.0,110.0,160.0,65.0,120.0,73.0,77.0,500.0,136.0,233.0,125.0,300.0,190.0,192.0,126.0,203.0,55.0,100.0,122.0,174.0,69.0,165.0,320.0,64.0,78.0,224.0,80.0,71.0,340.0,144.0,50.0,387.0,230.0,86.0,111.0,107.0,103.0,63.0,95.0,204.0,40.0,284.0,740.0,97.0,272.0,87.0,142.0,260.0,135.0,265.0,211.0,106.0,200.0,333.0,220.0,299.0,114.0,67.0,245.0,51.0,82.0,196.0,258.0,349.0,70.0,155.0,313.0,113.0,61.0,184.0,345.0,108.0,98.0,129.0,84.0,59.0,250.0,321.0,235.0,256.0,231.0,252.0,7515.0,197.0,280.0,41.0,185.0,232.0,187.0,396.0,43.0,2

In [11]:
cate_cols.map { col ->
    val data = trainDf[col].values.groupingBy { it }.eachCount()
    plot {
        line {
            x(data.keys, name = col)
            y(data.values, name = "price")
            color = Color.BLUE
        }
    }
}[0]

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="0neozE"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"model",
"y":"price"
},
"stat":"identity",
"data":{
"price":[2342.0,4502.0,927.0,386.0,543.0,412.0,3762.0,4496.0,9573.0,1460.0,6038.0,5052.0,2730.0,168.0,448.0,139.0,11762.0,4391.0,537.0,1872.0,920.0,8445.0,3827.0,811.0,508.0,3121.0,225.0,1349.0,270.0,572.0,2063.0,2195.0,96.0,464.0,965.0,395.0,1524.0,1469.0,2608.0,5186.0,1309.0,1177.0,1789.0,583.0,1696.0,1775.0,1522.0,318.0,49.0,290.0,196.0,301.0,2004.0,375.0,375.0,705.0,638.0,2454.0,337.0,349.0,201.0,1078.0,293.0,1020.0,662.0,45.0,430.0,598.0,111.0,467.0,202.0,284.0,249.0,675.0,123.0,468.0,1250.0,125.0,351.0,291.0,554.0,367.0,56.0,88.0,310.0,44.0,157.0,167.0,92.0,361.0,142.0,198.0,164.0,185.0,518.0,150.0,577.0,168.0,226.0,57.0,88.0,228.0,416.0,115.0,270.0,42.0,55.0,495.0,337.0,125.0,279.0,93.0,150.0,165.0,73.0,16.0,63.0,1084.0,344.0,205.0,171.0,85.0,287.0,372.0,62.0,170.0,227.0,101.0,142.0,296.0,265.0,62.0,126.0,112.0,238.0,64.0,588.0,312.0,143.0,147.0,199.0,193.0,325.0,121.0,261.0,68.0,199.0,289.0,26.0,218.0,91.0,28.0,169.0,77.0,182.0,171.0,120.0,17.0,52.0,286.0,566.0,10.0,250.0,162.0,161.0,87.0,109.0,148.0,55.0,76.0,17.0,95.0,134.0,185.0,55.0,144.0,89.0,43.0,48.0,47.0,98.0,168.0,151.0,109.0,325.0,86.0,61.0,141.0,102.0,125.0,51.0,237.0,144.0,164.0,129.0,126.0,142.0,149.0,13.0,200.0,52.0,35.0,260.0,7.0,345.0,106.0,164.0,102.0,122.0,2.0,65.0,60.0,95.0,68.0,23.0,37.0,56.0,68.0,59.0,146.0,135.0,30.0,52.0,4.0,14.0,67.0,3.0,49.0,103.0,89.0,47.0,74.0,50.0,26.0,18.0,11.0,23.0,16.0,38.0,18.0,2.0,20.0,10.0,10.0,1.0,7.0,2.0,2.0,1.0],
"model":[30.0,40.0,115.0,109.0,110.0,24.0,13.0,26.0,19.0,7.0,1.0,48.0,65.0,138.0,105.0,114.0,0.0,8.0,78.0,21.0,3.0,4.0,31.0,121.0,75.0,17.0,107.0,16.0,111.0,47.0,5.0,44.0,200.0,155.0,87.0,20.0,22.0,63.0,49.0,29.0,88.0,60.0,73.0,39.0,23.0,11.0,69.0,167.0,199.0,178.0,101.0,6.0,10.0,57.0,33.0,32.0,52.0,46.0,74.0,93.0,125.0,41.0,127.0,104.0,98.0,68.0,156.0,116.0,170.0,160.0,85.0,80.0,143.0,77.0,166.0,53.0,66.0,203.0,14.0,42.0,119.0,174.0,177.0,210.0,36.0,238.0,70.0,58.0,126.0,84.0,97.0,108.0,192.0,64.0,15.0,95.0,54.0,142.0,55.0,218.0,222.0,163.0,76.0,83.0,122.0,18.0,124.0,106.0,59.0,228.0,164.0,165.0,145.0,94.0,186.0,220.0,179.0,67.0,71.0,25.0,144.0,212.0,172.0,168.0,206.0,152.0,123.0,182.0,38.0,117.0,132.0,99.0,183.0,219.0,62.0,173.0,82.0,34.0,102.0,213.0,92.0,162.0,43.0,175.0,35.0,61.0,113.0,137.0,226.0,161.0,134.0,202.0,130.0,146.0,139.0,45.0,112.0,227.0,195.0,2.0,51.0,232.0,129.0,135.0,169.0,191.0,148.0,140.0,9.0,201.0,237.0,221.0,171.0,196.0,204.0,185.0,147.0,189.0,193.0,214.0,56.0,180.0,100.0,187.0,91.0,208.0,150.0,131.0,28.0,188.0,149.0,12.0,118.0,184.0,120.0,50.0,136.0,176.0,223.0,79.0,37.0,133.0,27.0,246.0,96.0,153.0,103.0,194.0,154.0,240.0,225.0,90.0,86.0,207.0,234.0,141.0,181.0,211.0,197.0,81.0,217.0,216.0,157.0,243.0,239.0,159.0,244.0,128.0,72.0,158.0,190.0,215.0,205.0,151.0,224.0,236.0,231.0,230.0,89.0,198.0,209.0,233.0,241.0,229.0,-1.0,235.0,245.0,242.0,247.0]
},
"color":"#5470c6",
"sampling":"none",
"position":"identity",
"geom":"line"
}]
};
 var plotContainer = document.getElementById("0neozE");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M23.31924920060519 325.5 L23.31924920060519 325.5 L25.19983381355722 15.499999999999943 L27.08041842650925 166.37492560156446 L28.961003039461282 317.9878836833602 L30.841587652413313 301.276719666695 L32.722172265

In [12]:
boolean_cols.map { col ->
    val data = trainDf[col].values.groupingBy { it }.eachCount()
    plot {
        line {
            x(data.keys, name = col)
            y(data.values, name = "price")
            color = Color.BLUE
        }
    }
}[0]

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="lY2psK"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"gearbox",
"y":"price"
},
"stat":"identity",
"data":{
"price":[111623.0,32396.0,5981.0],
"gearbox":[0.0,1.0,-1.0]
},
"color":"#5470c6",
"sampling":"none",
"position":"identity",
"geom":"line"
}]
};
 var plotContainer = document.getElementById("lY2psK");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 60,000 
 
 
 
 
 
 
 80,000 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 
 
 price 
 
 
 
 
 gearbox

对离散变量进行标签特征编码

In [13]:
class LabelEncoding() {
    val map = mutableMapOf<Double, Double>()
    var max = 0.0
    fun get(key: Double) =
        if (map.containsKey(key))
            map[key]
        else {
            max++
            map[key] = max
            max
        }
}

In [14]:
cate_cols.forEach { colName ->
    val labelEncoding = LabelEncoding()
    val col = trainDf.getColumn(colName)
    trainDf = trainDf.sortBy { col }.update { col }.with { labelEncoding.get(it as Double) }
}
cate_cols.forEach { colName ->
    val labelEncoding = LabelEncoding()
    val col = targetDf.getColumn(colName)
    targetDf = targetDf.sortBy { col }.update { col }.with { labelEncoding.get(it as Double) }
}

对数字变量进行归一化

In [15]:
num_cols.forEach { colName ->
    val col = trainDf.getColumn(colName)
    val min = col.minBy { it as Double } as Double
    val max = col.maxBy { it as Double } as Double
    trainDf = trainDf.update { col }.with { ((it as Double) - min) / (max - min) }
}
num_cols.forEach { colName ->
    val col = targetDf.getColumn(colName)
    val min = col.minBy { it as Double } as Double
    val max = col.maxBy { it as Double } as Double
    targetDf = targetDf.update { col }.with { ((it as Double) - min) / (max - min) }
}

获取数据并切割训练集与测试集

In [16]:
val data = trainDf.select(*(num_cols + cate_cols + boolean_cols).toTypedArray())
    .rows().map {
        it.toMap().map { it.value as Double }
    } to
        trainDf["price"].toList().map { listOf(it as Double) }

val (trainData, testData) = data.first.zip(data.second).shuffled().let {
    it.drop((it.size * 0.1).toInt()) to it.dropLast((it.size * 0.9).toInt())
}

In [17]:
val trainX = mk.ndarray(trainData.map { it.first })
val trainY = mk.ndarray(trainData.map { it.second })
val testX = mk.ndarray(testData.map { it.first })
val testY = mk.ndarray(testData.map { it.second })

定义模型

定义训练轮次

In [23]:
val model = Model(
    Dense((num_cols + cate_cols + boolean_cols).size, 128, initialize = HeUniformInitialize()),
    LeakyRelu(),
    Dropout(0.2),
    Dense(128, 256, initialize = HeUniformInitialize()),
    LeakyRelu(),
    Dropout(0.2),
    Dense(256, 256, initialize = HeUniformInitialize()),
    LeakyRelu(),
    Dropout(0.2),
    Dense(256, 256, initialize = HeUniformInitialize()),
    LeakyRelu(),
    Dropout(0.2),
    Dense(256, 128, initialize = HeUniformInitialize()),
    LeakyRelu(),
    Dropout(0.2),
    Dense(128, 1, initialize = HeUniformInitialize()),
    SoftPlus(base = 1.01,maxClip = 10000.0),
    loss = Mse(),
    optimizer = Adam(),
    scheduler = ExponentialScheduler(1e-2, dropRate = 1 - 1e-2)
)

In [24]:
val trainEpochs = (1..20)

开始训练！！！！

In [25]:
val lossList = mutableListOf<Double>()
val evaluateList = mutableListOf<Double>()

In [26]:
val loss = model.evaluate(trainX, trainY)
val evaluate = model.evaluate(testX, testY)
println("predect=${model.predict(testX)[0]} trueValue=${testY.get(0)}")
evaluateList.add(evaluate)
println("success train in 0 epochs, with evaluate=$evaluate")
println("-------------------------------------------------------------------------------------------")

predect=[69.62692056327121] trueValue=[6850.0]
success train in 0 epochs, with evaluate=9.01264827004627E7
-------------------------------------------------------------------------------------------


In [27]:
trainEpochs.forEach { 
    val loss = model.fitWithBatchSize(trainX, trainY, 100)
    lossList.add(loss)
    val evaluate = model.evaluate(testX, testY)
    println("predect=${model.predict(testX)[0]} trueValue=${testY.get(0)}")
    evaluateList.add(evaluate)
    println("success train in $it epochs, with loss=$loss and evaluate=$evaluate")
    println("-------------------------------------------------------------------------------------------")
}

predect=[69.68320607083213] trueValue=[6850.0]
success train in 1 epochs, with loss=9.059885393803729E7 and evaluate=9.00509361887814E7
-------------------------------------------------------------------------------------------
predect=[69.66022382006551] trueValue=[6850.0]
success train in 2 epochs, with loss=9.05988487648708E7 and evaluate=9.005092829976445E7
-------------------------------------------------------------------------------------------
predect=[69.65957874765537] trueValue=[6850.0]
success train in 3 epochs, with loss=9.059884547666517E7 and evaluate=9.005092614802937E7
-------------------------------------------------------------------------------------------
predect=[69.65972705396773] trueValue=[6850.0]
success train in 4 epochs, with loss=9.059884824582565E7 and evaluate=9.00509291182353E7
-------------------------------------------------------------------------------------------
predect=[69.66054290092745] trueValue=[6850.0]
success train in 5 epochs, with loss=9.0

org.jetbrains.kotlinx.jupyter.exceptions.ReplInterruptedException: The execution was interrupted

绘制 loss 曲线

In [28]:
dataFrameOf(
    "epochs" to lossList.mapIndexed { index, _ -> index } + evaluateList.mapIndexed { index, _ -> index },
    "loss" to lossList + evaluateList,
    "category" to lossList.map { "train" } + evaluateList.map { "test" }
).plot {
    line {
        x("epochs")
        y("loss")
        color("category")
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="UzqpxN"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"loss":[9.059885393803729E7,9.05988487648708E7,9.059884547666517E7,9.059884824582565E7,9.05988450268351E7,9.059884937419133E7,9.059885138328607E7,9.01264827004627E7,9.00509361887814E7,9.005092829976445E7,9.005092614802937E7,9.00509291182353E7,9.005092601444472E7,9.00509300788187E7,9.00509320145621E7],
"category":["train","train","train","train","train","train","train","test","test","test","test","test","test","test","test"],
"epochs":[0.0,1.0,2.0,3.0,4.0,5.0,6.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"epochs",
"y":"loss",
"color":"category"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
}]
};
 var plotContainer = document.getElementById("UzqpxN");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 90.1M 
 
 
 
 
 
 
 90.2M 
 
 
 
 
 
 
 90.3M 
 
 
 
 
 
 
 90.4M 
 
 
 
 
 
 
 90.5M 
 
 
 
 
 
 
 90.6M 
 
 
 
 
 
 
 
 
 loss 
 
 
 
 
 epochs 
 
 
 
 
 
 
 
 
 category 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 train 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 test

对目标进行预测并保存预测结果

In [27]:
val rows = targetDf.select(*(listOf("SaleID") + num_cols + cate_cols + boolean_cols).toTypedArray()).rows().map {
    val id = it.SaleID.toInt()
    val result = model.predictOne(mk.ndarray(it.values().drop(1).map { it as Double }))[0].toInt()
    listOf(id, result)
}
csvWriter().writeAll(rows, targetFileName = "res/result.csv")